# Оценка результатов А\В теста. Тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;

# Цели исследования: 

Провести оценку результатов A/B-теста, Оцените корректность проведения теста и проанализируйте его результаты, удалось ли за 14 дней с момента регистрации в системе увидеть что пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов

# Задачи: 

 - Обзор данных
 - Предобработка данных 
 - Оценка корректности проведения А\В теста и его соответсвие ТЗ
 - Исследовательский анализ данных
 - Проведение А\В теста
 - Выводы

# Обзор  данных

In [ ]:
#Загружаем библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import statistics as st
import plotly.graph_objects as go
from scipy import stats as st
from sklearn import tree, ensemble
from matplotlib import pyplot as plt
import plotly.express as px
from scipy.stats import spearmanr
from scipy.stats import iqr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
from plotly.subplots import make_subplots

In [ ]:
#Прочитаем файлы  и сохраним в переменные
try:
    new_users = pd.read_csv('C:\\Users\\Acer\\Downloads\\final_ab_new_users.csv', parse_dates=['first_date'], dayfirst=True)
    marketing_events = pd.read_csv('C:\\Users\\Acer\\Downloads\\ab_project_marketing_events.csv')
    participants = pd.read_csv('C:\\Users\\Acer\\Downloads\\final_ab_participants.csv')
    events = pd.read_csv('C:\\Users\\Acer\\Downloads\\final_ab_events.csv',parse_dates=['event_dt'], dayfirst=True)
except:
    new_users = pd.read_csv('/datasets/final_ab_new_users.csv',parse_dates=['first_date'], dayfirst=True)
    marketing_events = pd.read_csv('/datasets/ab_project_marketing_events.csv')
    participants = pd.read_csv('/datasets/final_ab_participants.csv')
    events = pd.read_csv('/datasets/final_ab_events.csv',parse_dates=['event_dt'], dayfirst=True)

In [ ]:
#ф-ция для обзора данных
def inspect(df):
    display(df.head()) 
    df.info() 
    print(50*'-')
    print ('Количество дубликатов -', df.duplicated().sum())
    print ('Кол-во пропусков:')
    print (df.isna().sum())
    
    display(df.describe())
    display()
    df.columns = [x.lower().replace(' ', '_') for x in df.columns.values]
    for column_name in df.columns:
            if 'start_dt' in column_name or 'finish_dt' in column_name:
                df[column_name] = pd.to_datetime(df[column_name])
    return df

new_users = inspect(new_users)

In [ ]:
marketing_events = inspect(marketing_events)

In [ ]:
participants = inspect(participants)

In [ ]:
events = inspect(events)

## Промежуточные выводы: 

У нас есть 4 таблицы с данными: 

1) marketing_events - календарь маркетинговых событий на 2020 год:
- name — название маркетингового события;
- regions — регионы, в которых будет проводиться рекламная кампания;
- start_dt — дата начала кампании;
- finish_dt — дата завершения кампании.

2) new_users - все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года:
- user_id — идентификатор пользователя;
- first_date — дата регистрации;
- region — регион пользователя;
- device — устройство, с которого происходила регистрация.

3) participants - Участники теста:
- user_id — идентификатор пользователя;
- ab_test — название теста;
- group — группа пользователя.
    
4) events - все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года:
- user_id — идентификатор пользователя;
- event_dt — дата и время события;
- event_name — тип события;
- details — дополнительные данные о событии. Например, для покупок,purchase , в этом поле хранится стоимость покупки в долларах.

Мы привели дату к нужному типу, привели столбцы к стилистически правильному формату, явных дубликатов выявлено не было. Было выявлено 377577 в таблице событий новых пользователей в столбце details.

# Предобработка данных

## Проверим данные на неявные дубликаты

In [ ]:
#выведем неявные дубликаты из разных датафреймов
print('Кол-во неявных дубликатов в new_users -',new_users['user_id'].duplicated().sum())
print(50*'-')
print('Кол-во неявных дубликатов в participants -',participants['user_id'].duplicated().sum())
print(50*'-')
print('Кол-во неявных дубликатов в events  -',events['user_id'].duplicated().sum())

In [ ]:
#Доля дубликатов
events['user_id'].duplicated().sum() / len(events) * 100

In [ ]:
#Проверим неявные дубликаты по нескольким признакам
events.pivot_table(index=['user_id','event_dt'],values='event_name',aggfunc=({'event_name':'unique'})).reset_index()

Можем заметить, что пользователи под одним id совершают разные действия в разное время, т.е для дальнейшего исследования убрать мы их не можем

В датафрейме events  более 86% не явных дубликатов

In [ ]:
#Доля дубликатов
participants['user_id'].duplicated().sum() / len(events) * 100

В датафрейме participants менее 1% не явных дубликатов

In [ ]:
#Проверим неявные дубликаты по нескольким признакам
participants.pivot_table(index=['user_id','group'],values='ab_test',aggfunc=({'ab_test':'unique'}))\
.reset_index().sort_values(['user_id','group'])

Можем заметить, что у нас присутствует несколько тестов. Из ТЗ мы знаем, что нас интересует [recommender_system_test]

In [ ]:
#Оставим только нужный для нас тест в новом ДТ
recommender_system_test = participants[participants['ab_test'] == 'recommender_system_test']
#Проверим неявные дубликаты
recommender_system_test['user_id'].duplicated().sum()

В нужных нам данных дубликаты отсутсвуют

In [ ]:
#Проверка пропусков
events.groupby('event_name')['details'].unique()

Можем заметить, что пропуски присутсвуют в категория логина,продуктовой карточки и продуктовой странички, нет пропусков в категории оплаты в долларах, можем сделать вывод, что этот столбец специально предназначен для оплаты,соответственно с этими пропусками сделать ничего мы не можем

# Оценим корректность проведения теста

## Техническое задание

Название теста: recommender_system_test ;

Группы: А (контрольная), B (новая платёжная воронка);

Дата запуска: 2020-12-07;

Дата остановки набора новых пользователей: 2020-12-21;

Дата остановки: 2021-01-04;

Ожидаемое количество участников теста: 15% новых пользователей из
региона EU;

Назначение теста: тестирование изменений, связанных с внедрением
улучшенной рекомендательной системы;

Ожидаемый эффект: за 14 дней с момента регистрации в системе
пользователи покажут улучшение каждой метрики не менее, чем на 5
процентных пунктов:

- конверсии в просмотр карточек товаров — событие product_page
- просмотры корзины — product_cart
- покупки — purchase .

## Выделим пользователей участвующих в тесте и проверим:

### Период набора пользователей в тест и его соответствие с требованием ТЗ

In [ ]:
#период набора пользователей
print('Начало набора пользователей -',new_users['first_date'].min())
print('Остановка набора пользователей -',new_users['first_date'].max())

Можем заметить,что есть пользователи, которые зарегестрировались после 21.12,можем их отсеять, т.к. это неудовлетворяет условиям ТЗ

In [ ]:
#Убираем пользователей которые сарегистрированы позже 21.12.2020
new_users = new_users[new_users['first_date'] <= '2020-12-21']
new_users

###  Регион регистрации пользователей: все ли попавшие в тест пользователи представляют целевой регион и составляет ли общее количество пользователей из целевого региона 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест;

In [ ]:
#Отберем пользователей из EU
new_users_eu = new_users[new_users['region'] == 'EU']
new_users_eu.head()

Соеденим таблицы участников теста с таблицей регистраций

In [ ]:
#Объединение таблиц
df = new_users_eu.merge(participants,on='user_id',how='left')
df

In [ ]:
#Проверим долю пользователей
df.query('ab_test == "recommender_system_test"').shape[0] / new_users_eu.shape[0]

количество пользователей из целевого региона составляет 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест как и указано в ТЗ

### Динамику набора пользователей в группы теста и проверьте равномерность распределения пользователей по группам теста и корректность их формирования;

In [ ]:
#Оставим только наш тест
df = df.query('ab_test == "recommender_system_test"')

In [ ]:
#Добавим в нашу прошлую объединенную таблицу данные из таблицы events
data = df.merge(events,on='user_id',how='left')

In [ ]:
#Кол-во пользователей в группах
print('Группа A -',data[data['group'] == 'A']['user_id'].nunique())
print('Группа B -',data[data['group'] == 'B']['user_id'].nunique())

In [ ]:
#Динамика набора пользователей по группам
df_1 = data.groupby(['group','first_date'])['user_id'].nunique().reset_index()
df_1


In [ ]:
#Строим линейный график
fig = px.line(df_1, x='first_date', y= 'user_id', title = 'Динамика набора пользователей по группам', \
             color = 'group',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Дата регистрации',
                  yaxis_title = 'Кол-во пользователей',
                  legend_title_text = 'Группы')
                  
fig.update_traces(hovertemplate = 'Дата : %{x} <br>пользователей: %{y}')

fig.show()

Можем заметить, что в больше всего регистраций в 1 день в середине теста 14 декабря и в последний день, по распределению группы поделены неравномерно, в Группе А в разрезе дней стабильно больше пользователей. Общее распределение Группа A - 3634,
Группа B - 2717

### (Опционально) оценим недельную цикличность набора пользователей в группы 

На предыдущем графике хорошо видно циклы, что в начале недели Пик по набору, так же в середине на 7 дней - Пик, и в последний день теста - Пик, цикл каждую новую неделю

## Изучим данные о пользовательской активности:

### Проверим даты совершения событий участниками теста: совпадают ли они с датами проведения теста, согласно техническому заданию;

In [ ]:
#Проверим совпадают ли даты события с датами проведения теста
print('Дата первого события -',events['event_dt'].min())
print('Дата последнего события -',events['event_dt'].max())

Дата первого события совпадает с началом набора новых пользователей, дата последнего события попадает в промежуток проведения теста, но тест заканчивается 04.01.21, а последнее событие происходит 30.12.20. Если учитывать что в ТЗ наш лайфтайм 14 дней, то даты совершения событий попадают в наш тест

### Проверим активность пользователей: все ли зарегистрированные пользователи прошли авторизацию и совершали переход по продуктовой воронке; если есть пользователи, которые не совершали событий после регистрации, изучим их количество и распределение между группами теста; сделаем вывод о необходимости учитывать пользователей без событий при изучении результатов теста;

In [ ]:
#Посмотрим распределение кол-ва действий по типам событий и группам
data_activ_A = data.query('group == "A"').groupby(['group','event_name'])['user_id'].nunique().reset_index()
data_activ_A = data_activ_A.reindex([0,2,1,3])
data_activ_B = data.query('group == "B"').groupby(['group','event_name'])['user_id'].nunique().reset_index()
data_activ_B = data_activ_B.reindex([0,2,1,3])
display(data_activ_A)
display(data_activ_B)

In [ ]:
#Построим воронку событий
fig = go.Figure(go.Funnel(
    y= data_activ_A['event_name'],
    x=data_activ_A['user_id']
    ))
fig.update_layout(title_text='Воронка событий группы А')
fig.show()


In [ ]:
#Построим воронку событий
fig = go.Figure(go.Funnel(
    y= data_activ_B['event_name'],
    x=data_activ_B['user_id']
    ))
fig.update_layout(title_text='Воронка событий группы В')
fig.show()


Мы видим что по воронкам, что и в группе А и в группе В воронка нарушена. Делают покупки  люди  чаще, чем в заходят в корзину.Скорее всего это связано с тем, что можно купить в 1 клик, без просмотра корзины товара.

In [ ]:
#Распределение пользователей по группам, которые не совершали никаких действий
data.\
query('event_name != "product_page" and event_name != "login" and event_name != "product_cart" and event_name != "purchase"').\
groupby('group')['user_id'].count().reset_index()


Пользователей которые не совершали каких либо действий после регистрации 2870 человек, 1030 в группе А и 1840 в группе В

In [ ]:
#Распределение пользователей по датам, которые не совершали никаких действий
data_0 = data.\
query('event_name != "product_page" and event_name != "login" and event_name != "product_cart" and event_name != "purchase"').\
groupby(['group','first_date'])['user_id'].count().reset_index().sort_values(by =['group','first_date'],ascending=True)
data_0

In [ ]:
#Строим линейный график
fig = px.line(data_0, x='first_date', y= 'user_id',\
             title = 'Распределение пользователей по датам, которые не совершали никаких действий', \
             color = 'group',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Дата регистрации',
                  yaxis_title = 'Кол-во пользователей',
                  legend_title_text = 'Группы')
                  
fig.update_traces(hovertemplate = 'Дата : %{x} <br>Событий: %{y}')
fig.show()

Больше всего пользователей не совершавших какое либо действие пришло 12 и 13 декабря,После 13 декабря пользователй в группе А, которые не совершали  каких либо действий  не наблюдается. В группе В после 13 числа сильный спад по пользователям без действий, но после начинается снова рост

In [ ]:
#Распределение пользователей по девайсам
data.groupby('device')['user_id'].count().reset_index().sort_values(by ='user_id',ascending=False)

In [ ]:
#Распределение пользователей по девайсам, которые не совершали никаких действий
data.\
query('event_name != "product_page" and event_name != "login" and event_name != "product_cart" and event_name != "purchase"').\
groupby('device')['user_id'].count().reset_index().sort_values(by ='user_id',ascending=False)

Самый высокий показатель "бездействия" у пользователей с андроидом,но это связано с тем, что пользователей Андроида впринципе больше 

In [ ]:
#выборка без учета бездействующих пользователй
data_drop = data[(data.event_name == 'product_page') | (data.event_name == 'login') | (data.event_name =='product_cart') | (data.event_name =='purchace')]

print('Группа A -',data_drop[data_drop['group'] == 'A']['user_id'].nunique())
print('Группа B -',data_drop[data_drop['group'] == 'B']['user_id'].nunique())

В группе В доля бездействущих занимает около половины пользователей в группе А около 17%. На данном этапе мы можем убрать этих пользователей, они не будут влиять на результат теста, т.к. не будут учитываться в воронке событий. Удалим их в дальнейшем при построении сводной таблицы

###  Проверим горизонт анализа: рассчитаем лайфтайм совершения события пользователем после регистрации, оставим только те события, которые были совершены в первые 14 дней с момента регистрации; проверим, что все участники теста имели возможность совершать события все 14 дней с момента регистрации, оценим когда пользователи совершают свои первые события каждого вида.

In [ ]:
#оставим только те события, которые были совершены в первые 14 дней с момента регистрации
data = data[(data['event_dt'] - timedelta(14)) < data['first_date']]
data

In [ ]:
#Кол-во уникальных пользователей
data['user_id'].nunique()

In [ ]:
#проверим все ли пользователи совершали события
data['event_name'].unique()

In [ ]:
#Только активные пользователи
print('Группа A -',data[data['group'] == 'A']['user_id'].nunique())
print('Группа B -',data[data['group'] == 'B']['user_id'].nunique())

Можем заметить, что в горизонт анализа 14 дней, все пользователи совершали какое-либо событие

In [ ]:
#Проверим когда пользователи совершают свои первые события каждого вида
data.pivot_table(index='user_id',values='first_date',columns='event_name',aggfunc=({'first_date':'first'})).\
reset_index().sort_values(by=['login','product_cart','product_page','purchase'],ascending=True)


Можем заметить, что пользователи совершают события каждого вида в основном сразу после регистрации

In [ ]:
#Настройка даты
date = data
date['event_dt'] = pd.to_datetime(date['event_dt']).dt.floor('d')
date

In [ ]:
date['lifetime'] = date['event_dt'] - date['first_date']
lifetime = date.groupby(['group','lifetime'])['event_name'].count().reset_index()

lifetime['lifetime'] = lifetime['lifetime'].astype(str)
lifetime

In [ ]:
#Строим линейный график
fig = px.line(lifetime, x='lifetime', y= 'event_name',\
             title = 'Кол-во событий на каждый день в лайфтайме', \
             color = 'group',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Дата регистрации',
                  yaxis_title = 'Кол-во cобытий',
                  legend_title_text = 'Группы')
                  
fig.update_traces(hovertemplate = 'Дата : %{x} <br>Событий: %{y}')

fig.show()

На графике можем заметить, что кол-во событий в группе А в 1 день регистрации превышает группу В практически в 3 раза, далее с каждым следующим днем активность снижается и к концу теста практически уравнивается.

## Развернутый вывод о соответствии теста требованиям технического задания и возможности получения достоверных результатов АБ-теста, исходя из базового показателя конверсии в 50%.

* Мы выяснили,что есть пользователи, которые зарегестрировались после 21.12,отсеяли их, т.к. это неудовлетворяет условиям ТЗ
* Мы выяснили, что  попавшие в тест пользователи представляющие целевой регион и составляющте общее количество пользователей из целевого региона 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест, это пункт отвечает требованию ТЗ
* Больше всего регистраций в 1 день в середине теста 14 декабря и в последний день, по распределению группы поделены неравномерно, в Группе А в разрезе дней стабильно больше пользователей. Общее распределение Группа A - 3634, Группа B - 2717
* В группе А и в группе В воронка нарушена. В корзину с товарами люди заходят чаще, чем в продуктовые карточки.Скорее всего это связано с тем, можно товар можно сразу добавить в корзину, без просмотра карточки товара.
* Пользователей которые не совершали каких либо действий после регистрации 2870 человек, 1030 в группе А и 1840 в группе В, больше всего пользователей не совершавших какое либо действие пришло 13 и 12 декабря и пользующися Андроидом
* Мы отсеяли пользователей которые совершали события после 14 дней с момента регистрации. После этого у нас не осталось пользователей, которые не совершали какое-либо действие, выяснили что пользователи совершают какое-либо действие в основном сразу после регистрации. 
* Разница набора группы А и группы В около 35%, это может сущетсвенно повлиять на возможности получения достоверных результатов
* При базовом показателе конверсии в 50 % и минимальном обнаруживаемом эффекте 5% в относительных значениях минимальный размер должен составить - 6,277 участников в каждой группе. Если рассмотреть при этих же показательях и абсолютных значениях, то размер выборки каждой группы должен составлять 1567 человек. В этом случае совпадает наша контрольная группа по кол-ву человек, но тестовая всё равно не попадает. Исходя из этого мы можем сказать, что возможности получения достоверных результатов в нашем тесте подвергаются сомнениям

# Исследовательский анализ данных

## Распределим количества событий на пользователя в разрезе групп теста: построим гистограмму распределения этой величины в разрезе групп и сравним её средние значения между собой у групп теста;

In [ ]:
#Посмотрим на гистограмму распределения событий по пользователям
data.groupby('user_id')['event_dt'].count().hist(color = 'red', edgecolor = 'black',figsize = (10,6),bins=25)
plt.title ('Распределение событий по пользователям')
plt.xlabel ('Количество событий')
plt.ylabel ('Частота cобытий');

Можем заметить,что чаще всего пользователи совершают 6 событий, чаще всего от 2 до 9 событий

In [ ]:
#Посмотрим на распределение по группам
data.query('group == "A"').groupby('user_id')['event_dt'].count().hist(label = 'Группа А', edgecolor = 'black',\
                                                                       color = 'red',figsize = (13,9),bins=12)
plt.title ('Распределение событий по пользователям в группах')
plt.xlabel ('Количество событий')
plt.ylabel ('Частота событий')

data.query('group == "B"').groupby('user_id')['event_dt'].count().hist(label = 'Группа В',edgecolor = 'black', \
                                                                       color='gray',figsize = (13,9),bins=12)
plt.title ('Распределение событий по пользователям в группах')
plt.xlabel ('Количество событий')
plt.ylabel ('Частота событий')

plt.show()

Можем заметить,что кол-во событий в группе А стабильно выше, чем в группе В, причем в значительной мере.
Посмотрим на среднее двух групп

In [ ]:
#Сравним две группы по среднему кол-ву событий
test_A = data.query('group == "A"').groupby('user_id')['event_dt'].count().reset_index()
print('Среднее кол-во событий группы А -',test_A['event_dt'].mean().round())

test_B = data.query('group == "B"').groupby('user_id')['event_dt'].count().reset_index()
print('Среднее кол-во событий группы B -',test_B['event_dt'].mean().round())

## Проверим динамику количества событий в группах теста по дням: изучим распределение числа событий по дням и сравним динамику групп теста между собой. 

In [ ]:
#Настройка даты
date = data
date['event_dt'] = pd.to_datetime(date['event_dt']).dt.floor('d')
date

In [ ]:
#динамика количества событий по датам 
date_1 = date.groupby('event_dt')['event_name'].count().reset_index()
date_1

In [ ]:
#Строим линейный график
fig = px.line(date_1, x='event_dt', y= 'event_name',\
             title = 'Динамика событий по дням',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Дата события',
                  yaxis_title = 'Кол-во cобытий',
                  legend_title_text = 'Группы')
                  
fig.update_traces(hovertemplate = 'Дата : %{x} <br>Событий: %{y}')

fig.show()

Начиная с 13 декабря видим резкий скачек в активности пользователей до 21 числа, затем довольно резкий спад

In [ ]:
#динамика количества событий по датам в разрезе групп
date = date.groupby(['group','event_dt'])['event_name'].count().reset_index()

date

In [ ]:
#Строим линейный график
fig = px.line(date, x='event_dt', y= 'event_name',\
             title = 'Динамика событий по дням в разрезе групп', \
             color = 'group',color_discrete_sequence = px.colors.sequential.Rainbow)
fig.update_layout(xaxis_title = 'Дата события',
                  yaxis_title = 'Кол-во cобытий',
                  legend_title_text = 'Группы')
                  
fig.update_traces(hovertemplate = 'Дата : %{x} <br>Событий: %{y}')

fig.show()

На графике можем заметить резкий скачек кол-ва событий 13 декабря в группе А, рост продолжался  до 21 числа,затем пошел на спад . В группе В кол-во событий было практически равномерно за время всего теста, начиная с 21 числа как и у группы А начался спад, хоть и менее резкий



Можно сделать вывод,глядя на график в разрезе групп, что скачек на общей динамике был из-за всплеска активности в группе А,динамика группы В идет довольно равномерно

## Убедимся, что время проведения теста не совпадает с маркетинговыми и другими активностями. Настроим автоматическую проверку, выдающую список событий, пересекающихся с тестом. При необходимости оценим воздействие маркетинговых событий на динамику количества событий.

In [ ]:
#Проверим маркетинговые активности совпадающие с нашим тестом
marketing_events[marketing_events['regions'].str.contains('EU')].query('start_dt >= "2020-12-07" or finish_dt >= "2020-12-07"')

Мы взяли отрезок с начала старта регистрация нашего теста  и проверили проведения маркетинговых активностий в эти даты и по нашему региону, в котором проводится тест. Можно сделать вывод, что по нашему лайфтайму в 14 дней, маркетинговые активности не совпадают. Christmas&New Year Promo - начинается после 21 декабря

## Продуктовая воронка: построим простые продуктовые воронки для двух групп теста с учетом логической последовательности совершения событий; изучим изменение конверсии в продуктовой воронке тестовой группы, по сравнению с контрольной: наблюдается ли ожидаемый эффект увеличения конверсии в группе В, относительно конверсии в группе А?

In [ ]:
#Подготовка данных к воронке событий
data_A = data.query('group == "A"').groupby('event_name')['user_id'].nunique().reset_index()
data_A = data_A.reindex([0,2,1,3])
data_A

In [ ]:
#Подготовка данных к воронке событий
data_B = data.query('group == "B"').groupby('event_name')['user_id'].nunique().reset_index()
data_B = data_B.reindex([0,2,1,3])
data_B

Создадим сводную таблицу, что бы показать в разре групп кол-во пользователей, конверсию пользователей от 1 шага, конверсию пользователей от шага к шагу и долю потерь пользователей

In [ ]:
#Создаем сводную таблицу количества событий по группам
table =  data.pivot_table(index='event_name', values='user_id', columns='group', aggfunc='nunique')

#Поменяем местами события как они должны быть
table = table.reindex(['login','product_page','product_cart','purchase'])

#Посчитаем конверсию от логина
table_2 = (table / table.loc['login',:]).round(2)

#Объединяем сводную таблицу с таблицей коверсии от логина
table_2 = table.merge(table_2, on='event_name')

#Поменяем названия в столбцах
table_2 = table_2.rename (columns= {'A_x': 'A_count', 'B_x': 'B_count','A_y': 'A_conv', 'B_y':'B_conv'})

#Cчитаем конверсию в шаг
table_3 = table / table.shift(1)
table_3 = table_3.round(2)

#Заменим пропуски 1
table_3 = table_3.fillna(1)

#Объединяем таблицу с кол-вом и конверсией с таблицей с конверсией в шаг
table_2 = table_2.merge(table_3, on='event_name')

#Потери при переходе
table_4 = table_3 - table_3.shift(1)
table_4 = table_4.round(2)

#Объединяем таблицу с кол-вом и конверсией и конверсией в шаг с таблицей с потерями
table_2 = table_2.merge(table_4,on='event_name')

#Переименуем столбцы
table_2 = table_2.rename (columns= {'A_x': 'A_conv_previous', 'B_x': 'B_conv_previous','A_y': 'A_loss', 'B_y':'B_loss'})

#Заменим пропуски в потерях на 0
table_2 = table_2.fillna(0)

table_2

In [ ]:
#Строим воронки событий
fig = make_subplots(rows=1, cols=2,subplot_titles=("Группа А","Группа В"))


fig.add_trace(
    go.Funnel(
    y = data_A['event_name'],
    x = data_A['user_id'],
    textposition = "inside",
    textinfo = "value+percent initial+percent previous",
    marker = {"color": "#1c1847"}
),
    row=1, col=1
)

fig.add_trace(
    go.Funnel(
    y = data_B['event_name'],
    x = data_B['user_id'],
    textposition = "inside",
    textinfo = "value+percent initial+percent previous",
        marker = {"color": "#ff0000"}
),
    row=1, col=2
)

fig.update_layout(showlegend=False,height=600, width=1000)
fig.show()

Глядя на воронки можем сказать, что конверсия в группе А лучше чем в тестовой группе В,в тестовой группе немного выше конверсия от продуктовой карточки до продуктовой корзины, но если рассмотреть конверсию к корзине от логина, то в группе А она все равно выше. Опять стоит обратить внимание, что пользователей совершивших покупку больше чем тех, которые переходят в корзину товара, это тенденция для двух групп остается не изменной. Наблюдаемый эффект увеличение конверсии для тестовой группы не наблюдается

## Общий вывод об изменении пользовательской активности в тестовой группе, по сравнению с контрольной.

В ходе исследования мы выяснили, что кол-во событий в группе А стабильно выше, чем в группе В, причем в значительной мере.
* Среднее кол-во событий группы А - 7.0
* Среднее кол-во событий группы B - 5.0

На графике динамики по дням мы заметили резкий скачек кол-ва событий у контрольной группы 13 декабря, к 16 декабря произошел легкий спад и до 20 числа распределение было примерно равномерным, после 20 числа пошел рост у тестируемой группы динамика довольно стабильна, +- в одном диапазоне

Мы взяли отрезок с начала сентября проведения маркетинговых активностий по которому сделали вывод, что по нашему лайфтайму в 14 дней, маркетинговые активности не совпадают. Christmas&New Year Promo - начинается после 21 декабря, Black Friday Ads Campaign заканчивается раньше проведения нашего теста, остальные активности не подходят по нашему региону

Построили воронки событий и проверили конверсию контрольной группы относительно тестовой где у контрольной группы конверсия:

* От продуктовой карточке к логину составила - 65%
* От корзины к продуктовой карточке составила - 46% , от корзины к логину - 30%
* От оплаты к корзине составила - 107% от оплаты к логину - 32% 

конверсия тестовой группы 

* От продуктовой карточке к логину составила - 56%
* От корзины к продуктовой карточке составила - 49% , от корзины к логину - 27.9%
* От оплаты к корзине составила - 102% от оплаты к логину - 28.4% 

Обратили внимание, что пользователей совершивших покупку больше чем тех, которые переходят в корзину товара, это тенденция для двух групп остается неизменной. Наблюдаемый эффект увеличение конверсии для тестовой группы не наблюдается

# Проведем оценку результатов A/B-тестирования:

## Проверим статистическую разницу долей z-критерием.

In [ ]:
#Подготовим данные к тесту
data_AA = data.groupby(['event_name','group'])['user_id'].nunique().reset_index().sort_values(by='group')
data_AA = data_AA.reindex([0,4,2,6,1,5,3,7])

data_AA


In [ ]:
#Функция для проведения Z-теста
def test(ab1, ab2, event, alpha, bon):  

    
    # критический уровень статистической значимости c поправкой Бонферрони
    bonferroni_alpha = alpha / bon
 
    # число пользователей в группе 1 и группе 2:
    trials = np.array([ab1['user_id'].nunique(), 
                       ab2['user_id'].nunique()])

    # число пользователей, совершивших событие в группе 1 и группе 2
    success = np.array([ab1[ab1['event_name'] == event]['user_id'].nunique(), 
                        ab2[ab2['event_name'] == event]['user_id'].nunique()])
    # пропорция успехов в первой группе:
    p1 = success[0]/trials[0]

    # пропорция успехов во второй группе:    
    p2 = success[1]/trials[1]
    
    # пропорция успехов в комбинированном датасете:
    p_combined = (success[0] + success[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2 

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference /  np.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)  

    p_value = (1 - distr.cdf(abs(z_value))) * 2   #тест двусторонний, удваиваем результат
    
    print('Событие:', event)
    print('p_value:', p_value)

    if p_value < bonferroni_alpha:
        print('Статистически значимые различия между долями есть - отвергаем нулевую гипотезу')
    else:
        print(
        'Статистически значимых различий между долями нет - не получилось отвергнуть нулевую гипотезу')

* Н0 - Статистически значимых различий между долями - нет
* Н1 - Статистически значимые различия между долями - есть
* Уровень значимости альфа = 0.05
* Воспользуемся поправкой Бонферрони

In [ ]:
# Уровень значимости:
alpha = 0.05

# Поправка Бонферрони для А/B теста:
bonB = 4

In [ ]:
# проверим, есть ли статистически значимая разница между контрольной группой А и тестовой В:

for event in data_AA['event_name'].unique():
    test(data[data['group'] == 'A'], data[data['group'] == 'B'], event, alpha, bonB)
    print()

## Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

Статистическая разница присутствует только в карточках товара(В пользу группы А), в остальных событиях не удалось выявить статистически значимых различий.  Ожидаемого эффеката достичь неудалось

# Общий вывод

Изначально данные по ТЗ были даны не корректно, в этапе предобрабокте мы убрали пользователей не из целевого региона, в наших данных были пользователи, которые зарегистрированы позже, чем указана последняя дата регистрации в ТЗ, так же у нас было два теста изначально, оставили тот, который был по ТЗ. Выянсли что количество пользователей из целевого региона составляет 15% от общего числа пользователей из целевого региона, зарегистрированных в период набора пользователей в тест как и указано в ТЗ. Больше всего регистраций в 1 день в середине теста 14 декабря и в последний день, по распределению группы поделены неравномерно, в Группе А в разрезе дней стабильно больше пользователей. Общее распределение Группа A - 3634, Группа B - 2717
В группе А и в группе В воронка нарушена. В корзину с товарами люди заходят чаще, чем в продуктовые карточки.Скорее всего это связано с тем, можно товар можно сразу добавить в корзину, без просмотра карточки товара.
Пользователей которые не совершали каких либо действий после регистрации 2870 человек, 1030 в группе А и 1840 в группе В, Больше всего пользователей не совершавших какое либо действие пришло 12 и 13 декабря,После 13 декабря пользователй в группе А, которые не совершали каких либо действий не наблюдается. В группе В после 13 числа сильный спад по пользователям без действий, но после начинается снова рост и самый высокий показатель "бездействия" у пользователей с андроидом,но это связано с тем, что пользователей Андроида впринципе больше
Мы отсеяли пользователей которые совершали события после 14 дней с момента регистрации. После этого у нас не осталось пользователей, которые не совершали какое-либо действие, выяснили что пользователи совершают какое-либо действие в основном сразу после регистрации.
Разница набора группы А и группы В около 35%, это может сущетсвенно повлиять на возможности получения достоверных результатов
При базовом показателе конверсии в 50 % и минимальном обнаруживаемом эффекте 5% минимальный размер выборки составил 1567

В ходе исследования мы выяснили, что кол-во событий в группе А стабильно выше, чем в группе В, причем в значительной мере.
На графике можем заметить резкий скачек кол-ва событий 13 декабря в группе А, рост продолжался до 21 числа,затем пошел на спад . В группе В кол-во событий было практически равномерно за время всего теста, начиная с 21 числа как и у группы А начался спад, хоть и менее резкий.
Пересечений с маркетинговыми активностим замечено не было.
Построили воронки событий и проверили конверсию контрольной группы относительно тестовой и выяснили что показатели контрольной группы визуально показываютс себя лучше
Проверили статистическую разницу долей z-критерием и выяснили что статистическая разница между группами А и В есть только в карточке товаров.

Тест можно считать завершенным, данные не соответствовали ТЗ, группы были набраны слишком неравномерно, что и повлияло на результаты теста.